In [1]:
import pandas as pd
from collections import Counter

In [2]:
from ast import literal_eval
from bs4 import BeautifulSoup

In [38]:
from matplotlib import pyplot as plt

In [4]:
from tqdm._tqdm_notebook import tqdm_notebook

C:\Users\alecr\Anaconda3\envs\gensim_env\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  """Entry point for launching an IPython kernel.


In [5]:
tqdm_notebook.pandas()

In [6]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [7]:
df = pd.read_csv(r"D:\data\openparliament\statements_nospeaker_en.csv")
df.drop('Unnamed: 0',axis=1, inplace=True)

C:\Users\alecr\Anaconda3\envs\gensim_env\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (20,23,28,29,30,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [54]:
politician_full = pd.read_csv(r"D:\data\openparliament\politician_full.csv")
politician_full.drop(['Unnamed: 0'], axis=1,inplace=True)

In [8]:
df.head()

,id,politician_id,riding_id,party_id,end_date,start_date,id.1,document_id,time,h1_en,...,slug,urlcache,h1_fr,h2_fr,h3_fr,who_fr,who_context_fr,wordcount_en,tokenized_content,slug_length
0,2611,3465,35066,4,2005-11-29,1997-09-22,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,marlene-catterall-1,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3
1,3210,3465,70224,4,1997-04-27,1994-01-17,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,marlene-catterall-1,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3
2,4305,173,70358,1,\N,2015-10-19,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,tom-lukiwski-7,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3
3,1534,173,47007,1,2015-10-19,2004-10-04,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,tom-lukiwski-7,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3
4,1541,43,47014,1,2015-10-19,2004-10-04,313253,1621,2004-02-06 10:05:00-05,NaN,...,garry-breitkreuz-1,/debates/2004/2/6/garry-breitkreuz-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'today', ...",3


In [9]:
%%time
#takes about five minutes, and progress_apply is a bit slower
texts = df.tokenized_content.apply(literal_eval)

  0%|          | 0/861398 [00:00<?, ?it/s]

Wall time: 7min 39s


In [10]:
df['tokenized_content'] = texts

In [31]:
word_counts = dict()

In [32]:
template_counter = Counter()

In [33]:
pids = list(set(df.politician_id))

In [34]:
for pol in pids:
    word_counts[pol] = template_counter.copy()

In [15]:
tdf = df[:1000]

In [35]:
def update_counts(row):
    tokens = row.loc['tokenized_content']
    if word_counts.get(row.loc['politician_id'])==None:
        word_counts[row.loc['politician_id']] = template_counter.copy()
        word_counts[row.loc['politician_id']].update(tokens)
    else:
        word_counts[row.loc['politician_id']].update(tokens)

In [17]:
%%time
tdf.apply(update_counts, axis=1)

Wall time: 116 ms


0      None
1      None
2      None
3      None
4      None
       ... 
995    None
996    None
997    None
998    None
999    None
Length: 1000, dtype: object

In [36]:
%%time
df.progress_apply(update_counts, axis=1)

  0%|          | 0/861398 [00:00<?, ?it/s]

Wall time: 1min 6s


0         None
1         None
2         None
3         None
4         None
          ... 
861393    None
861394    None
861395    None
861396    None
861397    None
Length: 861398, dtype: object

In [29]:
import pickle

In [37]:
with open('D:\data\openparliament\models\word_counts.pkl','wb') as f:
    pickle.dump(word_counts,f)

In [19]:
#word_counts[173]

In [20]:
#word_counts[3465]

In [22]:
c.most_common

<bound method Counter.most_common of Counter({'the': 51168, ',': 50274, 'to': 32326, '.': 30792, 'of': 24100, 'that': 22774, 'and': 16454, 'in': 15632, 'a': 14516, 'i': 13384, 'is': 12396, 'be': 11242, 'for': 8578, 'it': 7968, 'speaker': 7762, 'have': 7652, 'we': 7266, 'this': 7220, 'not': 7208, 'would': 7028, 'mr.': 6992, 'on': 6048, 'government': 4998, 'all': 4642, 'was': 4422, 'are': 4264, 'they': 4022, 'bill': 3970, 'as': 3946, 'with': 3942, 'by': 3766, 'will': 3668, 'member': 3446, 'or': 3292, 'questions': 3224, 'members': 3222, 'from': 3196, 'my': 3186, 'an': 3126, 'if': 3090, "'s": 3064, 'has': 2878, 'there': 2688, 'he': 2668, 'house': 2664, 'ask': 2662, 'do': 2626, 'stand': 2534, '(': 2422, ')': 2422, 'allowed': 2376, 'order': 2286, 'what': 2262, 'their': 2256, 'at': 2242, 'one': 2224, 'no': 2142, 'but': 2102, 'minister': 2042, 'which': 2022, 'committee': 2022, 'who': 1986, 'could': 1942, 'should': 1936, 'question': 1922, 'been': 1896, 'because': 1884, 'standing': 1854, 'made':

In [23]:
doc = nlp("Mr. Speaker, I rise on a point of order. I believe you would find unanimous consent following consultation among all parties to further defer the vote just deferred until Monday to next Tuesday at the end of government orders.")

In [24]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Speaker 4 11 PERSON
Monday 171 177 DATE
next Tuesday 181 193 DATE


In [26]:
%%time
df['text_en'] = df.content_en.progress_apply(lambda x: BeautifulSoup(x).text)

  0%|          | 0/861398 [00:00<?, ?it/s]

Wall time: 3min 37s


In [27]:
I'm going to have to do the spacy processing in chunks so I can still use my computer. 

SyntaxError: EOL while scanning string literal (<ipython-input-27-86ffe04ee84a>, line 1)

In [ ]:
spacy.prefer_gpu()

In [ ]:
%%time
nlp_objects = list(nlp.pipe(df.text_en[50000:100000]))

In [ ]:
nlp_dict = dict()

In [ ]:
nlp_dict['nlp'] = nlp_objects

save nlp_objects to a file here:

In [ ]:
import pickle

In [ ]:
with open(r'D:\data\openparliament\spacy_data\nlp_chunk_3.pkl','wb') as f:
    pickle.dump(nlp_dict,f)

In [ ]:
utt = df.pivot_table(values='content_en',index='politician_id',aggfunc='count')

In [ ]:
utt.sort_values(by='content_en',ascending=False,inplace=True)

In [ ]:
utt.head()

In [ ]:
df['wordcount']= df.tokenized_content.apply(lambda x: sum(x))

In [40]:
punctuation = '!"#$%&\'()*+,-./:;<=>@[\\]^_`{|}~'
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [41]:
c = word_counts[173]

In [43]:
words = c.most_common(50)

In [47]:
words = [w for w in words if not w[0] in stop_words]
words = [w for w in words if not w[0] in punctuation]

In [48]:
words

[('speaker', 7750),
 ('would', 6998),
 ('mr.', 6982),
 ('government', 4964),
 ('bill', 3970),
 ('member', 3428),
 ('questions', 3220),
 ('members', 3214),
 ("'s", 3052),
 ('ask', 2658),
 ('house', 2628),
 ('stand', 2530)]

In [91]:
for key, counter in word_counts.items():
    name = politician_full[politician_full.politician_id==key]['name'].values[0]
        
    print('politician_id: ', key, 'name: ', name)
    words = counter.most_common(100)
    words = [w for w in words if not w[0] in stop_words]
    words = [w for w in words if not w[0] in punctuation]
    print('most common words:', words)
    print('\n')
    
        
    

politician_id:  407 name:  Jean-Pierre Blackburn
most common words: [('mr.', 662), ('speaker', 629), ('canada', 521), ('veterans', 442), ('would', 438), ('quebec', 363), ('government', 360), ('people', 298), ('workers', 291), ('economic', 286), ('member', 279), ('support', 275), ('help', 249), ('also', 243), ('want', 234), ('new', 231), ('?', 219), ('bloc', 218), ('like', 206), ("'s", 204), ('work', 201), ('labour', 201), ('weeks', 199), ('bill', 195), ('million', 194), ('members', 192), ('canadian', 191), ('us', 188), ('two', 186), ('measures', 175), ('development', 174), ('house', 173), ('regions', 173), ('time', 168), ('\\n', 167), ('years', 159), ('employment', 154), ('benefits', 152), ('take', 149), ('québécois', 147), ('one', 142), ('minister', 140), ('act', 140)]


politician_id:  1 name:  Jim Abbott
most common words: [('would', 4492), ('government', 4160), ('canada', 3948), ('minister', 3396), ('people', 3396), ('bill', 3100), ('speaker', 2880), ("'s", 2880), ('member', 2720),

most common words: [('government', 990), ('would', 879), ('minister', 669), ('speaker', 591), ('mr.', 576), ("'s", 534), ('bill', 462), ('band', 453), ('fisheries', 450), ('?', 408), ('tsawwassen', 396), ('treaty', 381), ('members', 372), ('issue', 351), ('one', 336), ('question', 330), ('home', 324), ('people', 318), ('parliament', 315), ('act', 312), ('member', 300), ('regulations', 300), ('department', 291), ('canada', 288), ('fishery', 288), ('committee', 288), ('house', 285), ('british', 282), ('fraser', 279), ('fish', 264), ('columbia', 252), ('reserve', 249), ('canadian', 249), ('year', 240)]


politician_id:  10321 name:  Nicola Di Iorio
most common words: [('would', 102), ('canada', 79), ('member', 69), ('government', 69), ('speaker', 66), ("'s", 64), ('house', 58), ('people', 56), ('one', 54), ('mr.', 53), ('country', 52), ('also', 50), ('canadians', 46), ('time', 44), ('like', 42), ('members', 39), ('colleague', 38), ('work', 37), ('rcmp', 37), ('thank', 35), ('colleagues', 

most common words: [('government', 13308), ('bill', 11364), ('quebec', 9999), ('would', 8568), ('canada', 8016), ('bloc', 6243), ('minister', 5856), ("'s", 5037), ('people', 4974), ('one', 4677), ('federal', 4626), ('québécois', 4566), ('house', 4509), ('us', 4428), ('speaker', 3933), ('must', 3879), ('mr.', 3747), ('industry', 3732), ('want', 3723), ('liberal', 3591), ('?', 3534), ('money', 3516), ('time', 3333), ('could', 3231), ('members', 3216), ('colleague', 3180), ('conservatives', 3177), ('party', 3153), ('act', 3069), ('member', 3009), ('conservative', 2985), ('make', 2976)]


politician_id:  159 name:  Francine Lalonde
most common words: [('canada', 2046), ('government', 1956), ('?', 1852), ('minister', 1808), ('would', 1758), ('mr.', 1712), ('speaker', 1634), ('international', 1506), ("'s", 1452), ('quebec', 1422), ('must', 1136), ('people', 1120), ('bill', 1118), ('one', 1026), ('countries', 928), ('also', 868), ('say', 848), ('could', 842), ('time', 820), ('said', 818), ('u

most common words: [('government', 4521), ('quebec', 4138), ('would', 2609), ('minister', 2576), ('bill', 2248), ('canada', 2167), ("'s", 1939), ('federal', 1685), ('mr.', 1609), ('speaker', 1545), ('tax', 1384), ('also', 1356), ('canadian', 1340), ('house', 1307), ('?', 1223), ('bloc', 1203), ('one', 1153), ('us', 1064), ('must', 1055), ('trade', 1044), ('prime', 1031), ('said', 980), ('like', 960), ('member', 944), ('time', 941), ('billion', 936), ('people', 925), ('provinces', 906), ('made', 873), ('international', 822), ('québécois', 815), ('know', 766), ('years', 753), ('motion', 752)]


politician_id:  4317 name:  Elsie Wayne
most common words: [('minister', 643), ('government', 594), ('?', 491), ('would', 473), ('house', 463), ('speaker', 436), ('mr.', 417), ('people', 417), ('canada', 355), ('us', 348), ('one', 317), ("'s", 281), ('member', 253), ('defence', 246), ('want', 245), ('today', 243), ('canadian', 241), ('know', 232), ('time', 231), ('hon', 228), ('prime', 218), ('say

most common words: [('bill', 435), ('would', 279), ('government', 229), ('speaker', 227), ('mr.', 222), ('canada', 186), ('like', 176), ('colleague', 165), ('?', 160), ('people', 135), ("'s", 116), ('thank', 115), ('also', 106), ('rights', 103), ('member', 100), ('\\n', 98), ('want', 97), ('criminal', 92), ('conservatives', 81), ('canadian', 81), ('ndp', 81), ('must', 81), ('committee', 77), ('police', 73), ('said', 71), ('work', 70), ('hon', 70), ('support', 70), ('one', 67), ('act', 65), ('canadians', 65), ('make', 64), ('safety', 63), ('human', 63), ('justice', 63), ('development', 63), ('federal', 62), ('minister', 61)]


politician_id:  297 name:  Robert Vincent
most common words: [('government', 579), ('would', 473), ('?', 473), ('bill', 368), ('quebec', 353), ('workers', 325), ('people', 303), ('industry', 283), ('speaker', 267), ('mr.', 251), ('canada', 251), ("'s", 229), ('one', 218), ('money', 182), ('like', 181), ('member', 173), ('us', 172), ('think', 170), ('want', 160), (

...

most common words: [('women', 288), ('canada', 267), ('would', 222), ("'s", 159), ('speaker', 158), ('canadians', 130), ('also', 129), ('government', 129), ('mr.', 117), ('people', 99), ('world', 94), ('bill', 93), ('canadian', 92), ('house', 90), ('work', 85), ('like', 83), ('international', 83), ('make', 82), ('ottawa', 79), ('many', 78), ('committee', 77), ('today', 76), ('support', 73), ('one', 72), ('member', 70), ('us', 68), ('hon', 67), ('time', 66), ('country', 65), ('important', 65), ('rights', 65), ('new', 64), ('could', 64), ('treaty', 64), ('minister', 62), ('every', 61), ('need', 61), ('human', 60), ('working', 59), ('members', 58), ('proud', 58), ('thank', 57), ('want', 57)]


politician_id:  2728 name:  Liza Frulla
most common words: [('mr.', 438), ('speaker', 308), ('canada', 266), ('canadian', 224), ('also', 196), ('women', 140), ('would', 140), ('minister', 140), ('cultural', 138), ('one', 134), ('must', 132), ("'s", 130), ('said', 128), ('first', 118), ('government',

most common words: [('government', 3938), ('would', 3916), ('member', 2718), ('?', 2266), ('minister', 2212), ('speaker', 2146), ('mr.', 1868), ('canada', 1854), ("'s", 1702), ('bill', 1610), ('canadians', 1588), ('like', 1474), ('also', 1470), ('many', 1466), ('people', 1454), ('one', 1382), ('tax', 1226), ('us', 1222), ('members', 1220), ('canadian', 1188), ('prime', 1172), ('know', 1166), ('liberal', 1108), ('new', 1096), ('time', 1030), ('support', 980), ('trade', 954), ('important', 932), ('liberals', 904), ('place', 880), ('make', 852), ('work', 804)]


politician_id:  3033 name:  Randy White
most common words: [('government', 3740), ('would', 2644), ('house', 2588), ('?', 2140), ('people', 2088), ('one', 1844), ('speaker', 1740), ('get', 1472), ('bill', 1452), ('time', 1440), ('sex', 1424), ('mr.', 1404), ('country', 1336), ('minister', 1292), ('canada', 1272), ("'s", 1168), ('like', 1144), ('want', 1036), ('offender', 1028), ('said', 1004), ('registry', 972), ('say', 960), ('le

politician_id:  9380 name:  Simon Marcil
most common words: [('government', 528), ('quebec', 417), ('speaker', 345), ('mr.', 291), ('workers', 279), ('?', 243), ("'s", 213), ('minister', 201), ('would', 174), ('federal', 168), ('people', 156), ('canada', 135), ('want', 111), ('producers', 105), ('liberals', 105), ('one', 99), ('us', 99), ('work', 96), ('supply', 96), ('bloc', 96), ('québécois', 96), ('house', 96), ('bill', 96), ('time', 93), ('right', 93), ('even', 93), ('management', 93), ('dairy', 90), ('vote', 90), ('liberal', 84), ('ottawa', 81), ('get', 81), ('quebeckers', 81), ('question', 78), ('like', 75), ('legislation', 75), ('colleague', 72), ('french', 72), ('problem', 69), ('rights', 69)]


politician_id:  9382 name:  Kerry Diotte
most common words: [('would', 89), ("'s", 88), ('canadians', 80), ('speaker', 75), ('government', 69), ('canada', 68), ('bill', 66), ('liberals', 61), ('?', 61), ('mr.', 58), ('people', 55), ('crimean', 51), ('know', 43), ('tatars', 42), ('today'

most common words: [('government', 478), ('canada', 404), ('speaker', 313), ('public', 297), ('mr.', 275), ('member', 261), ('pay', 260), ("'s", 214), ('would', 195), ('colleague', 173), ('system', 172), ('people', 169), ('canadians', 167), ('gatineau', 162), ('work', 155), ('new', 142), ('us', 131), ('hon', 127), ('also', 114), ('tax', 113), ('?', 112), ('house', 110), ('minister', 110), ('canadian', 110), ('servants', 109), ('like', 107), ('phoenix', 106), ('thank', 105), ('want', 105), ('one', 104), ('service', 103), ('budget', 103), ('support', 101), ('employees', 101), ('party', 98), ('years', 94), ('every', 91), ('time', 90), ('know', 89), ('working', 86)]


politician_id:  9666 name:  Diane Lebouthillier
most common words: [('tax', 750), ('speaker', 456), ('mr.', 447), ('government', 362), ('cra', 290), ('canadians', 268), ('canada', 249), ('agency', 226), ('evasion', 194), ('revenue', 171), ('billion', 163), ('information', 153), ('committed', 140), ("'s", 138), ('year', 122), 

politician_id:  3849 name:  Stéphane Bergeron
most common words: [('government', 4152), ('would', 2613), ('house', 2238), ('speaker', 2187), ('mr.', 1956), ('minister', 1851), ("'s", 1689), ('?', 1680), ('canada', 1566), ('quebec', 1563), ('bill', 1485), ('motion', 1260), ('one', 1206), ('members', 1203), ('us', 1125), ('must', 1065), ('member', 1065), ('federal', 1065), ('said', 1020), ('people', 1017), ('also', 987), ('time', 981), ('could', 909), ('colleague', 864), ('made', 822), ('bloc', 816), ('canadian', 810), ('committee', 810), ('say', 786), ('order', 753), ('like', 744), ('public', 726), ('since', 696), ('party', 690), ('prime', 690)]


politician_id:  9994 name:  Georgina Jolibois
most common words: [('indigenous', 334), ('people', 327), ('government', 325), ('first', 236), ('speaker', 198), ('canada', 191), ('communities', 176), ('northern', 173), ('saskatchewan', 167), ('?', 167), ('mr.', 163), ('would', 160), ('nations', 156), ('bill', 122), ('métis', 111), ("'s", 109), (

In [55]:
politician_full.head()


,id,politician_id,riding_id,party_id,end_date,start_date,name,short_name,slug
0,4015,2693,70159,5,2000-09-11,2000-04-02,André Harvey,André,Harvey
1,4016,2693,70159,4,2001-01-28,2000-09-12,André Harvey,André,Harvey
2,2676,2693,70159,26,2000-04-01,1997-09-22,André Harvey,André,Harvey
3,1774,2693,24016,4,2004-05-23,2001-01-29,André Harvey,André,Harvey
4,4017,4136,59023,4,2005-05-23,2002-01-28,Joe Peschisolido,Joe,Peschisolido


In [83]:
name = politician_full[politician_full.politician_id==173]['name']

In [84]:
name.index[0]

1053

In [89]:
politician_full[politician_full.politician_id==18]['name'].values[0]


'Sue Barnes'